In [1]:
from datetime import date, datetime
import json
from pathlib import Path
from random import randint

from django.db import models, transaction, IntegrityError
from django.urls import reverse
import numpy as np
import pandas as pd


# ORM queries in jupyter notebook env
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

%load_ext autoreload
%autoreload 2

home = Path.home()
shared = Path(home, 'shared')
ranch_tools = Path(home/'site/ranch-tools/ranch_tools')

In [13]:
breeding_season = 2023
previous_breeding_season = breeding_season - 1

In [22]:
# Annotate the queryset with cow birth year, and create pandas DataFrame of
# queryset sorted by breeding seaons, and cow birth year

pregchecks_queryset = PregCheck.objects.annotate(cow_birth_year=models.F('cow__birth_year'))
pc_df = pd.DataFrame(list(pregchecks_queryset.values())).sort_values(['breeding_season', 'cow_birth_year'])
pc_df['cow_birth_year'] = pc_df.cow_birth_year.fillna(0).astype('int')

In [30]:
# Create group by df of cow birth year, and breeding season

# first groupby df is just cow birth year, and breeding season
group_by_cols = ['cow_birth_year', 'breeding_season']
by_bs_grouped = pc_df.groupby(group_by_cols).size().reset_index(name='all_count')

# second groupby df is same as above but filtered by is_pregnant == True, and another by is_pregnant == False
is_preg = pc_df.is_pregnant == True
by_bs_preg_grouped = pc_df[is_preg].groupby(group_by_cols).size().reset_index(name='preg_count')
by_bs_preg_grouped['preg_count'] = by_bs_preg_grouped['preg_count'].astype('int')
by_bs_open_grouped = pc_df[~is_preg].groupby(group_by_cols).size().reset_index(name='open_count')
by_bs_open_grouped['open_count'] = by_bs_open_grouped['open_count'].astype('int')

# third groupby df is same as above but filtered by is_recheck == True, and another by is_recheck == False
is_recheck = pc_df.recheck == True
by_bs_preg_recheck_grouped = pc_df[is_preg & is_recheck].groupby(group_by_cols).size().reset_index(name='preg_recheck_count')
by_bs_preg_recheck_grouped['preg_recheck_count'] = by_bs_preg_recheck_grouped['preg_recheck_count'].astype('int')
by_bs_open_recheck_grouped = pc_df[~is_preg & is_recheck].groupby(group_by_cols).size().reset_index(name='open_recheck_count')
by_bs_open_recheck_grouped['open_recheck_count'] = by_bs_open_recheck_grouped['open_recheck_count'].astype('int')


In [39]:
# merge all the above groupby dfs to createa df with all the desired count columns:
# all_count', 'preg_count', 'open_count', 'preg_recheck_count', 'open_recheck_count

mdf1 = pd.merge(by_bs_grouped, by_bs_preg_grouped, on=group_by_cols, how='left')
mdf1['preg_count'] = mdf1['preg_count'].fillna(0).astype('int')

mdf2 = pd.merge(mdf1, by_bs_open_grouped, on=group_by_cols, how='left')
mdf2['open_count'] = mdf2['open_count'].fillna(0).astype('int')

mdf3 = pd.merge(mdf2, by_bs_preg_recheck_grouped, on=group_by_cols, how='left')
mdf3['preg_recheck_count'] = mdf3['preg_recheck_count'].fillna(0).astype('int')

mdf4 = pd.merge(mdf3, by_bs_open_recheck_grouped, on=group_by_cols, how='left')
mdf4['open_recheck_count'] = mdf4['open_recheck_count'].fillna(0).astype('int')

count_cols = ['all_count', 'preg_count', 'open_count', 'preg_recheck_count', 'open_recheck_count']
mdf4[count_cols] = mdf4[count_cols].fillna(0)

In [44]:
by_bs_counts_df = mdf4.sort_values(['breeding_season', 'cow_birth_year'], ascending=[False, False])
by_bs_counts_df['age'] = 0
has_age = by_bs_counts_df.cow_birth_year > 0
by_bs_counts_df.loc[has_age, 'age'] = by_bs_counts_df[has_age].breeding_season - by_bs_counts_df[has_age].cow_birth_year



In [47]:
by_bs_counts_df['first_pass_pregnant'] = by_bs_counts_df.preg_count - by_bs_counts_df.preg_recheck_count
by_bs_counts_df['first_pass_open'] = by_bs_counts_df.open_count - by_bs_counts_df.open_recheck_count
by_bs_counts_df['first_pass_open_plus_pregnant'] = by_bs_counts_df.first_pass_open + by_bs_counts_df.first_pass_pregnant


In [48]:
by_bs_counts_df['net_pregnant'] = by_bs_counts_df.preg_count
by_bs_counts_df['net_open'] = by_bs_counts_df.first_pass_open - by_bs_counts_df.preg_recheck_count


In [49]:
by_bs_counts_df['ending_inventory'] = by_bs_counts_df.net_pregnant + by_bs_counts_df.net_open

In [50]:
by_bs_counts_df['preg_pct'] = by_bs_counts_df.net_pregnant / by_bs_counts_df.ending_inventory * 100
by_bs_counts_df['preg_pct'] = by_bs_counts_df['preg_pct'].map(lambda x: round(x))

In [52]:
by_bs_counts_df['net_open_less_first_pass_open'] = by_bs_counts_df.net_open - by_bs_counts_df.first_pass_open
by_bs_counts_df.head(5)

,cow_birth_year,breeding_season,all_count,preg_count,open_count,preg_recheck_count,open_recheck_count,age,first_pass_pregnant,first_pass_open,first_pass_open_plus_pregnant,net_pregnant,net_open,ending_inventory,preg_pct,net_open_less_first_pass_open
49,2022,2023,5,5,0,0,0,1,5,0,5,5,0,5,100,0
48,2021,2023,203,200,3,0,0,2,200,3,203,200,3,203,99,0
47,2020,2023,203,126,77,1,0,3,125,77,202,126,76,202,62,-1
44,2019,2023,198,165,33,2,2,4,163,31,194,165,29,194,85,-2
41,2018,2023,197,186,11,1,0,5,185,11,196,186,10,196,95,-1


In [53]:
this_year_by_bs_counts_df = by_bs_counts_df[by_bs_counts_df.breeding_season == breeding_season]
last_year_by_bs_counts_df = by_bs_counts_df[by_bs_counts_df.breeding_season == previous_breeding_season]



In [62]:
report_five_df = pd.merge(
    this_year_by_bs_counts_df, last_year_by_bs_counts_df[['cow_birth_year', 'age', 'ending_inventory']],
    how='left', on=['cow_birth_year'], suffixes=(f'_{breeding_season}', f'_{previous_breeding_season}')
)
report_five_df[f'age_{previous_breeding_season}'] = report_five_df[f'age_{previous_breeding_season}'].fillna(-1).astype('int')
report_five_df[f'ending_inventory_{previous_breeding_season}'] = report_five_df[f'ending_inventory_{previous_breeding_season}'].fillna(-1).astype('int')

report_five_df.head(5)

,cow_birth_year,breeding_season,all_count,preg_count,open_count,preg_recheck_count,open_recheck_count,age_2023,first_pass_pregnant,first_pass_open,first_pass_open_plus_pregnant,net_pregnant,net_open,ending_inventory_2023,preg_pct,net_open_less_first_pass_open,age_2022,ending_inventory_2022
0,2022,2023,5,5,0,0,0,1,5,0,5,5,0,5,100,0,-1,-1
1,2021,2023,203,200,3,0,0,2,200,3,203,200,3,203,99,0,-1,-1
2,2020,2023,203,126,77,1,0,3,125,77,202,126,76,202,62,-1,2,217
3,2019,2023,198,165,33,2,2,4,163,31,194,165,29,194,85,-2,3,187
4,2018,2023,197,186,11,1,0,5,185,11,196,186,10,196,95,-1,4,188


In [63]:
report_five_df['inventory_diff_from_prev_year'] = report_five_df[f'ending_inventory_{breeding_season}'] - report_five_df[f'ending_inventory_{previous_breeding_season}']



In [64]:
report_5_cols = [
    'cow_birth_year', f'age_{previous_breeding_season}', f'ending_inventory_{previous_breeding_season}', f'age_{breeding_season}',
    'first_pass_pregnant', 'first_pass_open', 'first_pass_open_plus_pregnant', 'preg_recheck_count',
    'net_pregnant', 'net_open', f'ending_inventory_{breeding_season}', 'preg_pct', 'net_open_less_first_pass_open', 'inventory_diff_from_prev_year'
]

final_report_five_df = report_five_df[report_5_cols]

final_report_five_df

,cow_birth_year,age_2022,ending_inventory_2022,age_2023,first_pass_pregnant,first_pass_open,first_pass_open_plus_pregnant,preg_recheck_count,net_pregnant,net_open,ending_inventory_2023,preg_pct,net_open_less_first_pass_open,inventory_diff_from_prev_year
0,2022,-1,-1,1,5,0,5,0,5,0,5,100,0,6
1,2021,-1,-1,2,200,3,203,0,200,3,203,99,0,204
2,2020,2,217,3,125,77,202,1,126,76,202,62,-1,-15
3,2019,3,187,4,163,31,194,2,165,29,194,85,-2,7
4,2018,4,188,5,185,11,196,1,186,10,196,95,-1,8
5,2017,5,201,6,207,7,214,2,209,5,214,98,-2,13
6,2016,6,189,7,182,10,192,1,183,9,192,95,-1,3
7,2015,7,132,8,146,3,149,3,149,0,149,100,-3,17
8,2014,8,142,9,149,8,157,1,150,7,157,96,-1,15
9,2013,9,112,10,85,7,92,1,86,6,92,93,-1,-20
